## Futbol Data Compilation

#### Import data from database

In [1]:
## Define connection to MySQL db

import MySQLdb
import pandas as pd
from pandas import DataFrame


def db_connect(tablename):
    # Connect
    db = MySQLdb.connect(host="127.0.0.1",
                         user="root",
                         passwd="omgilr",
                         db="futbol")

    cursor = db.cursor()

    # Execute SQL select statement
    cursor.execute("SELECT * FROM futbol.%s" % (tablename))  # change table name here

    field_names = [i[0] for i in cursor.description]
    get_data = [xx for xx in cursor]

    cursor.close()
    db.close()

    # Save as dataframe
    df = DataFrame(get_data)
    df.columns = field_names
    # # Close the connection
    # db.close()

    return df

#import db_connect # fix module object not callable error

In [2]:
# Create dataframes

tbl_list = ['events2015', 'players2015', 'leagues', 'teams']

events_df = db_connect(tbl_list[0])
players_df = db_connect(tbl_list[1])
leagues_df = db_connect(tbl_list[2])
teams_df = db_connect(tbl_list[3])

In [3]:
events_df.shape, players_df.shape, leagues_df.shape, teams_df.shape

((308631, 19), (150545, 71), (35, 3), (415, 4))

In [4]:
events_df.head(2)

,id,match_id,whoscored_id,competition,name,team_id,date,team,opponent,play_at,minute,second,type,start_x,start_y,outcome_type,is_touch,is_shot,qualifiers
0,1,959564,14260,EPL,Sergio Agüero,167,2016-01-02,Manchester City,Watford,away,0,1,Pass,50.0,50.0,Successful,1,0,"[\r\n {\r\n ""type"": {\r\n ""value"": 21..."
1,2,959564,97692,EPL,Raheem Sterling,167,2016-01-02,Manchester City,Watford,away,0,2,Pass,51.7,52.0,Successful,1,0,"[\r\n {\r\n ""type"": {\r\n ""value"": 14..."


#### Data extraction 

In [5]:
# CHOOSE THE DATAFRAME AND COLUMN THAT NEEDS TO BE PARSED

pars_df = events_df
pars_col = 'qualifiers'

In [6]:
import re
import ast
import collections
from collections import defaultdict
import numpy as np

# WHAT A FUCKER

dshape = pars_df.shape[0]

# Combine into dataframe

d = defaultdict(list) # Create dictionary to obtain unique columns
dl = [] # Create list to obtain values


# Eliminate aggressive syntax
for row in range(0,dshape):
    
    if pars_df[pars_col][row]:
        cc = pars_df[pars_col][row].replace('\r\n', '') # get rid of newlines syntax
        bcc = ast.literal_eval(cc) # get clean list of lists
        
        ls = []

        for i in range(0, len(bcc)):

            myString = str(bcc[i])

            # Get column name
            x = myString[myString.find("'displayName':"):-2]
            xs = x.split(",")
            y = xs[0]
            col = y[1:].split(":")[1].strip()
            col = col.replace("'", "")

            # Get value
            r = myString.split(",")[0]
            val = r[1:].split(":")[1].strip()
            

            if val == "{'displayName'": # if no value associated with term ignore
                pass
            else:
                ls.append([col, val])

        for i in range(0, len(ls)):
            d[ls[i][0]].append([ls[i][1], row])
            dl.append([ls[i][0], str(''.join(map(str,ls[i][1]))), row])


In [7]:
# Convert list of values to dataframe

dldf = pd.DataFrame(dl, columns = ['col_name', 'val', 'idx'])
dldf.tail(3)

,col_name,val,idx
1402655,PassEndY,'96.3',308630
1402656,Angle,'5.9',308630
1402657,PassEndX,'47.0',308630


In [8]:
# Create empty dataframe with dictionary keys

somelist = list()
for k,v in d.items():
    somelist.append(k)
    
raw_df = pd.DataFrame(columns = somelist)
raw_df['idx'] = range(0, max(dldf.idx)+1) # set up index in raw stats df
raw_df.tail(3)

,GoalDisallowed,SmallBoxLeft,KeeperSaved,KeyPass,Hands,LowLeft,FreekickTaken,OtherBodyPart,Length,BigChance,...,OutOfBoxCentre,DeepBoxRight,LeftFoot,ShotAssist,RegularPlay,OwnGoal,FromCorner,DirectFreekick,HighRight,idx
308628,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308628
308629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308629
308630,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,308630


In [9]:
### Find values in dldf col name equal to those in raw_stats_df

tc = len(raw_df.columns) - 1 # all the columns
mc = max(raw_df.idx + 1) # all the rows

for i in range(0, tc): # grab all columns (unique parameters)
    tdf = pd.DataFrame(dldf.loc[dldf['col_name'] == str(raw_df.columns[i]), ['val', 'idx']]) # col_name = raw_stats_df column, grab values and index
    tdf.columns = [raw_df.columns[i], 'idx'] 
    tdf = tdf.drop_duplicates(subset='idx', keep='last') # drop rows with multiple values per index, keep last entered value
    #tdf.set_index('idx') # set index
    new_index = pd.Index(range(0, mc, 1)) # fill in missing indexes to establish same column length
    #tdf = tdf.set_index('idx').reindex(new_index) # reset with full index
    cdf = pd.DataFrame(index = new_index)
    cdf['idx'] = cdf.index
    ndf = pd.merge(cdf, tdf, how = 'left', on = 'idx') # merge based on idx
    raw_df[raw_df.columns[i]] = ndf[ndf.columns[1]] # replace column
    print(i, len(ndf))
    

0 308631
1 308631
2 308631
3 308631
4 308631
5 308631
6 308631
7 308631
8 308631
9 308631
10 308631
11 308631
12 308631
13 308631
14 308631
15 308631
16 308631
17 308631
18 308631
19 308631
20 308631
21 308631
22 308631
23 308631
24 308631
25 308631
26 308631
27 308631
28 308631
29 308631
30 308631
31 308631
32 308631
33 308631
34 308631
35 308631
36 308631
37 308631
38 308631
39 308631
40 308631
41 308631
42 308631
43 308631
44 308631
45 308631
46 308631
47 308631
48 308631
49 308631
50 308631
51 308631
52 308631
53 308631
54 308631
55 308631
56 308631
57 308631
58 308631
59 308631
60 308631
61 308631
62 308631
63 308631
64 308631
65 308631
66 308631
67 308631
68 308631
69 308631
70 308631
71 308631
72 308631
73 308631
74 308631
75 308631
76 308631
77 308631
78 308631
79 308631
80 308631
81 308631
82 308631
83 308631
84 308631
85 308631
86 308631
87 308631
88 308631
89 308631
90 308631
91 308631
92 308631
93 308631
94 308631
95 308631
96 308631
97 308631
98 308631
99 308631
100 308631

In [10]:
raw_df.shape

(308631, 104)

In [13]:
# Remove single quotes

for i in range(1,104):

    raw_df[raw_df.columns[i]] = raw_df.iloc[:,i].map(lambda x: str(x).strip('\''))

In [14]:
raw_df.tail()

,GoalDisallowed,SmallBoxLeft,KeeperSaved,KeyPass,Hands,LowLeft,FreekickTaken,OtherBodyPart,Length,BigChance,...,OutOfBoxCentre,DeepBoxRight,LeftFoot,ShotAssist,RegularPlay,OwnGoal,FromCorner,DirectFreekick,HighRight,idx
308626,NaN,nan,nan,nan,nan,nan,nan,nan,28.4,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,308626
308627,NaN,nan,nan,nan,nan,nan,nan,nan,22.2,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,308627
308628,NaN,nan,nan,nan,nan,nan,nan,nan,37.1,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,308628
308629,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,308629
308630,NaN,nan,nan,nan,nan,nan,nan,nan,10.2,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,308630


#### Store data

In [15]:
# Store the dataframes for future sessions (csv)

string = pars_col + '_df'

raw_df.to_csv(string+'.csv', sep = ',')


In [ ]:
from pandas import HDFStore
import h5py

# HDF5 solution

#hdf = pd.HDFStore('extras2.h5') #create h5 file

#hdf = h5py.File('extras2', 'w')

#hdf[string] = raw_df  # save it


In [100]:
#hdf[string]  # load it

In [101]:
#hdf.close()